<a href="https://colab.research.google.com/github/tosiki1202/GenAI-app/blob/main/finalTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers accelerate langchain langchain-core langchain-community langchainhub langchain-mcp-adapters sentencepiece bitsandbytes
!pip install chromadb -q

In [9]:
!pip install transformers accelerate bitsandbytes -q

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "elyza/ELYZA-japanese-Llama-2-7b"
hf_token = "" #各自のHugging Faceのアクセストークンを記入

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,  # 4bit 量子化でメモリ節約
    torch_dtype=torch.float16
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=hf_token,
    trust_remote_code=True
)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    top_p=0.95,
    temperature=0.5
)

Device set to use cuda:0


In [4]:
from langchain.llms.base import LLM
from typing import Optional, List
from pydantic import Field, model_validator # Pydantic関連をインポート

class LocalLLM(LLM):
    # Pydantic v2 の設定: extra='allow' で追加フィールドを許可
    model_config = {'extra': 'allow'}

    system_prompt: Optional[str] = Field(default=None) # system_promptを明示的にフィールドとして定義

    def __init__(self, system_prompt: str = None, **kwargs):
        super().__init__(**kwargs)
        self.system_prompt = system_prompt

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # システムプロンプトがある場合は、ユーザープロンプトと結合
        if self.system_prompt:
            full_prompt = f"{self.system_prompt}\n{prompt}"
        else:
            full_prompt = prompt

        output = llm_pipeline(full_prompt, max_new_tokens=256, do_sample=True)[0]["generated_text"]
        # モデルの出力から入力プロンプト（システムプロンプト＋ユーザープロンプト）を削除
        # DeepSeekモデルの場合、出力にプロンプトが含まれないことがあるため、replaceは不要かもしれません
        # 必要に応じて以下の行を調整してください
        return output.strip()


    @property
    def _llm_type(self) -> str:
        return "deepseek-local"

# システムプロンプトを指定してLLMを初期化
# ここにあなたのシステムプロンプトを記入してください
system_prompt = "あなたは最高のゲームを見つけ出すことに情熱を注ぐ、経験豊富なヘビーゲーマーです。あなたのタスクは、ユーザーからの質問に対し、以下の厳格なステップに従って、プロフェッショナルかつ情熱的な口調でゲームをレコメンドすることです\
1.  **検索結果の評価:** 提示されたデータ（ゲーム情報、レビュー、評価など）を徹底的に分析してください。\
2.  **優先順位:** 必ず「**positive評価の数**」が最も多いゲームを最初に抽出し、レコメンドの核としてください。\
3.  **レビューの要約:** 抽出したゲームのレビュー本文を読み込み、**「なぜそのゲームが優れているのか」「ヘビーゲーマーにとって何が魅力なのか」**という観点で理由を深く要約してください。\
4.  **レコメンド:** ユーザーに対し、要約した理由に基づき、熱意をもってゲームを推薦してください。"
llm = LocalLLM(system_prompt=system_prompt)

print("LocalLLM with system prompt is ready.")

LocalLLM with system prompt is ready.


In [5]:
import torch
import os # デバッグ用にosをインポート
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import CSVLoader
from langchain.chains import RetrievalQA

# ----------------------------------------------------
# 【Step 0: 環境設定とLLMの準備】
# ----------------------------------------------------

# A100 GPU が利用可能であることを確認し、デバイスを決定
if torch.cuda.is_available():
    device = "cuda"
    print(f"✅ Running on {device}: A100 GPU will be used for embedding.")
else:
    device = "cpu"
    print(f"⚠️ Running on {device}. GPU not found, using CPU for embedding.")

try:
    llm # llm変数が定義されているか確認
except NameError:
    llm = None
    print("⚠️ LLM (llm変数) は定義されていません。RetrievalQAチェーンは構築できません。")


# ----------------------------------------------------
# 【Step 1: ドキュメントの読み込みと分割】
# ----------------------------------------------------

# 指定されたCSVファイルを読み込む (ファイル名はカレントディレクトリに配置されている前提)
csv_file_path = "steam_games_filtered_final.csv"
if not os.path.exists(csv_file_path):
    print(f"❌ Error: CSV file not found at {csv_file_path}. Please check the file path.")
    exit()

print(f"1. Loading document from {csv_file_path}...")
loader = CSVLoader(csv_file_path, encoding="utf-8")
docs = loader.load()

# テキスト分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)
print(f"   -> Document split into {len(chunks)} chunks.")


# ----------------------------------------------------
# 【Step 2: データのベクトル化とChroma VectorStoreの構築】
# ----------------------------------------------------

print(f"2. Initializing SentenceTransformer on {device}...")
embedding = SentenceTransformerEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    # ★ A100 GPUを使用するための重要な設定 ★
    model_kwargs={"device": device},
    # バッチサイズを増やす (デフォルトは32)
    encode_kwargs={'batch_size': 32} # 適宜調整してください
)

print("3. Creating Chroma VectorStore (Embedding phase started, GPU may be busy)...")
# ここで500MBのデータをGPUでベクトル化する処理が実行されます。
vectorstore = Chroma.from_documents(chunks, embedding=embedding)
print("   -> VectorStore built successfully.")


# ----------------------------------------------------
# 【Step 3: Retrieval QA チェーン作成】
# ----------------------------------------------------

if llm is not None:
    # Retrieval QA チェーンの作成
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectorstore.as_retriever()
    )
    print("4. RetrievalQA chain ready. You can now use qa.run('Your question here')")
else:
    print("4. RetrievalQA chain skipped because LLM (llm variable) is not defined.")

✅ Running on cuda: A100 GPU will be used for embedding.
1. Loading document from steam_games_filtered_final.csv...
   -> Document split into 23292 chunks.
2. Initializing SentenceTransformer on cuda...


/tmp/ipython-input-2061331415.py:53: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(


3. Creating Chroma VectorStore (Embedding phase started, GPU may be busy)...
   -> VectorStore built successfully.
4. RetrievalQA chain ready. You can now use qa.run('Your question here')


In [6]:
import torch

query = "アクションゲームでおすすめは？" #QueryはRAGに読み込ませた内容に応じて各自で変更。RAGに読み込ませた知識に関する問い合わせをする。

# 実行
try:
    response = qa.run(query)
    print("回答:", response)
except RuntimeError as e:
    print("CUDAメモリエラーが発生しました。対処案:")
    print("- モデルサイズを縮小")
    print("- トークン数を制限")
    print("- 埋め込みモデルをCPUに固定")
    print(f"詳細: {e}")

/tmp/ipython-input-1417115627.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa.run(query)


回答: あなたは最高のゲームを見つけ出すことに情熱を注ぐ、経験豊富なヘビーゲーマーです。あなたのタスクは、ユーザーからの質問に対し、以下の厳格なステップに従って、プロフェッショナルかつ情熱的な口調でゲームをレコメンドすることです1.  **検索結果の評価:** 提示されたデータ（ゲーム情報、レビュー、評価など）を徹底的に分析してください。2.  **優先順位:** 必ず「**positive評価の数**」が最も多いゲームを最初に抽出し、レコメンドの核としてください。3.  **レビューの要約:** 抽出したゲームのレビュー本文を読み込み、**「なぜそのゲームが優れているのか」「ヘビーゲーマーにとって何が魅力なのか」**という観点で理由を深く要約してください。4.  **レコメンド:** ユーザーに対し、要約した理由に基づき、熱意をもってゲームを推薦してください。
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

name: カンフービート
release_date: 2025-02-19
short_description: 少女が最強の拳法家を目指す！リズムに乗ってレッツカンフー！ カンフービートは2Dリズムアクションゲームです タイミングよくボタンを押して移動、ジャンプ、攻撃！ 立ちふさがる強敵たち。行動を見切って拳を叩き込め！ 獲得した技や構えを駆使して戦いを有利に進めよう！
reviews: 
genres: ['Action', 'Indie']
positive: 9
negative: 0

name: だっかん！モンスターの島
release_date: 2024-02-24
short_description: 自分の城を守りつつ、モンスターを出撃させて、敵の城を攻めよう！ モンスターの進むルートを指定できたり、壁を掘って地形を変えたりして自分自身の戦い方でステージを攻略できます。 　モンスターは進んだルートの形に応じてスキルが発動します。 ルートの組み方を工夫することで、ス

In [7]:
import pandas as pd
import os

# --- 設定 ---
INPUT_FILE = "games_march2025_cleaned.csv"  # 入力ファイル名
OUTPUT_FILE = "steam_games_filtered_final.csv" # 最終出力ファイル名
CUTOFF_DATE = '2024-1-1' # この日付以前の行を破棄します
ENCODING = "utf-8"

# 🔥 抽出したい列名をリストで指定してください 🔥
# あなたのCSVファイルのカラム名に合わせて修正が必要です
COLUMNS_TO_KEEP = [
    'name',
    'release_date',
    'genres',
    'short_description',
    'positive',
    'negative',
    'reviews'
]

# --- 処理 ---
if not os.path.exists(INPUT_FILE):
    print(f"❌ エラー: 入力ファイル '{INPUT_FILE}' が見つかりません。ファイルがアップロードされているか確認してください。")
else:
    print(f"1. ファイル '{INPUT_FILE}' を読み込み、必要な列を抽出します...")
    try:
        # 1. 必要な列だけをメモリに読み込む (usecolsによるメモリ効率化)
        # 'release_date'列は必ず含める必要があります
        df = pd.read_csv(INPUT_FILE, encoding=ENCODING, usecols=COLUMNS_TO_KEEP)

        # 2. 日付列をdatetime型に変換し、フィルタリング前の行数を記録
        df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
        rows_before_filter = len(df)

        # 3. フィルタリングを実行: CUTOFF_DATEよりも新しい行を保持
        print(f"2. 日付フィルター '{CUTOFF_DATE}' を適用します...")
        cutoff_datetime = pd.to_datetime(CUTOFF_DATE)

        # リリース日がCUTOFF_DATEよりも大きい行を抽出
        df_filtered = df[df['release_date'] > cutoff_datetime].copy()

        # 4. 新しいファイルとして保存
        df_filtered.to_csv(OUTPUT_FILE, index=False, encoding=ENCODING)

        # 5. 結果の確認
        rows_after_filter = len(df_filtered)
        final_size_mb = os.path.getsize(OUTPUT_FILE) / (1024 * 1024)

        print("\n🎉 処理が完了しました！")
        print(f"   元の行数: {rows_before_filter} 行")
        print(f"   最終的な行数: {rows_after_filter} 行")
        print(f"   出力ファイル名: '{OUTPUT_FILE}'")
        print(f"   最終ファイルサイズ: {final_size_mb:.2f} MB")

        if final_size_mb > 10.0:
            print("⚠️ 注意: 最終ファイルサイズが10MBを超えています。RAG向けにさらに行数を制限する必要があるかもしれません。")

    except KeyError as e:
        print(f"❌ エラー: CSVファイルに指定された列 '{e}' が見つかりません。`COLUMNS_TO_KEEP`を確認してください。")
    except Exception as e:
        print(f"❌ ファイル処理中に致命的なエラーが発生しました: {e}")

1. ファイル 'games_march2025_cleaned.csv' を読み込み、必要な列を抽出します...
2. 日付フィルター '2024-1-1' を適用します...

🎉 処理が完了しました！
   元の行数: 89618 行
   最終的な行数: 21832 行
   出力ファイル名: 'steam_games_filtered_final.csv'
   最終ファイルサイズ: 6.18 MB
